In [ ]:
# PAR

In [ ]:
%matplotlib inline
%autosave 5

In [ ]:
# necessary modules -------------------------------------------------------------------------------------
import pyfesom2 as pf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import time
from datetime import date

import cartopy.crs as ccrs
import cartopy.feature as cfeature

# own py_recom modules ----------------------------------------------------------------------

!jupyter nbconvert --to=python plot_Taylor_normalized.ipynb

!jupyter nbconvert --to=python plot_Chlorophyll_FESOM_python3.ipynb
from plot_Chlorophyll_FESOM_python3 import plot_Chl


In [ ]:
# After changing these parameters here, save the notebook and clear the output. 
# Then run this cell and the cells with the desired parameters.

# run specification -------------------------------------------------------------------------------------   
if True:
    runid      =  'fesom'
    layerwise    = False
    resultpath = '/work/ollie/mozeisin/results/f2r1.2/par_output'
    meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

    # period of analysis 
    first_year = 1948
    last_year  = 1949
    
    
years = np.arange(first_year,last_year+1,1)

# specification of analysis ------------------------------------------------------------------------------------
depth = 0
layerwise = False
mapproj = 'pc'
#Map projection. Options are Mercator (merc), Plate Carree (pc),
#North Polar Stereo (np), South Polar Stereo (sp),  Robinson (rob)
#Robinson projection is quite time consuming.

# export of analysis ------------------------------------------------------------------------------------
today = date.today().strftime("_%Y_%m_%d")

savepath = '/home/ollie/mozeisin/evaluation/mo_files/'
savefig = False


# just to check ------------------------------------------------------------------------------------
print('Processing years {2} to {3}\n\nReading out of {0}\nStoring graphs to {1}'.format(
    resultpath, savepath,years[0],years[1]))

In [ ]:
def mygrid(m,grid=False):
    '''
    Draw cartopy natural earth feature to axis m
    '''
    #m.coastlines(resolution='110m', color='black', linewidth=1)
    land_poly = cfeature.NaturalEarthFeature('physical', 'land', '110m',edgecolor='k',facecolor=cfeature.COLORS['land'])
    m.add_feature(land_poly)

    if grid:
        g1 = m.gridlines(draw_labels = True)
        g1.xlabels_top = False
        g1.xlabel_style = {'size': 16, 'color': 'gray'}
        g1.ylabel_style = {'size': 16, 'color': 'gray'}

In [ ]:
# mesh initialization -----------------------------------------------------------
mesh = pf.load_mesh(meshpath)

# PAR

In [ ]:
# loading 
label='PAR, mean over {0}-{1}'.format(first_year,last_year)
units = 'PAR [W m$^{-2}$]'

PAR = pf.get_data(resultpath, "PAR", 
                   #years, 
                   [1948],
                   mesh, runid=runid, how="mean", compute=True,silent=False)

PARsurf = pf.get_data(resultpath, "PAR", 
                   #years, 
                   [1948],
                   mesh, depth = 0, runid=runid, how="mean", compute=True,silent=False)

In [ ]:
# plotting
fig = pf.plot(mesh, PARsurf, mapproj = mapproj, units=units, titles=label, 
              levels = (0,150,31),
              cmap='viridis')

# Transect

In [ ]:
# Atlantic transect ------------------------------------------------------------------------------
lon_start = -25
lat_start = -80
lon_end = -25
lat_end = 80
npoints   = 50
lonlat = pf.transect_get_lonlat(lon_start, lat_start, lon_end, lat_end, npoints)

print('Plotting transect at lat {0} to {1}, lon {2} to {3}'.format(lat_start,lat_end,lon_start,lon_end))
pf.plot_transect_map(lonlat, mesh)

dist, tfPAR = pf.get_transect(PAR,mesh,lonlat)


pf.plot_xyz(mesh, 
            [tfPAR],
            dist,
            maxdepth=150,
            ncols=3,
            xlabel='Distance',
            label=['PAR [W m$^{-2}$]'],
            title=['Atlantic transect ({0}-{1})\nFESOM2-REcoM PAR'.format(first_year, last_year),],
            levels=np.linspace(0,150,31),
            cmap='viridis',
            figsize=(15, 6))